In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx

import time
import csv

import torch
import pyro

pyro.set_rng_seed(101)

In [155]:
# create generic discrete probability function
class cg_node():
    def __init__(self,n_inputs,name):
        
        self.n_inputs = n_inputs
        self.name = name
        
        if n_inputs == 0:
            self.label = 'exogenous'
        else:
            self.label = 'endogenous'
            
        return
    
    def p_init(self,input_data,var_data):
        
        self.n_data = len(input_data)
        
        self.input_data = input_data
        self.var_data = var_data
        
        if self.n_inputs == 0:
            p_ave = np.zeros(3)
            n_count = self.n_data
            for i in range(0,3):
                p_ave[i] = np.sum(var_data == i-1)/n_count
        
        elif self.n_inputs == 1:
            n_count = np.zeros(3)
            p_ave = np.zeros((3,3))
            
            for i in range(0,3):
                n_count[i] = np.sum(input_data == i-1)
                for j in range(0,3):
                    p_ave[j,i] = np.sum((input_data[:,0] == i-1)*(var_data == j-1))/n_count[i]
            
            
        elif self.n_inputs == 2:
            n_count = np.zeros((3,3))
            p_ave = np.zeros((3,3,3))
            
            for i in range(0,3):
                for j in range(0,3):
                    n_count[i,j] = np.sum((input_data[:,0] == i-1)*(input_data[:,1] == j-1))
                    for k in range(0,3):
                        p_ave[k,i,j] = np.sum(
                            (input_data[:,0] == i-1)*(input_data[:,1] == j-1)*(var_data == k-1))/n_count[i,j]
                        
        elif self.n_inputs == 3:
            n_count = np.zeros((3,3,3))
            p_ave = np.zeros((3,3,3,3))
            
            for i in range(0,3):
                for j in range(0,3):
                    for k in range(0,3):
                        n_count[i,j,k] = np.sum(
                            (input_data[:,0] == i-1)*(input_data[:,1] == j-1)*(input_data[:,2] == k-1))
                        for m in range(0,3):
                            p_ave[m,i,j,k] = np.sum((input_data[:,0] == i-1)*(input_data[:,1] == j-1)
                                *(input_data[:,2] == k-1)*(var_data == m-1))/n_count[i,j,k]
                            
        elif self.n_inputs == 4:
            n_count = np.zeros((3,3,3,3))
            p_ave = np.zeros((3,3,3,3,3))
            
            for i in range(0,3):
                for j in range(0,3):
                    for k in range(0,3):
                        for m in range(0,3):
                            n_count[i,j,k,m] = np.sum((input_data[:,0] == i-1)*(input_data[:,1] == j-1)
                                *(input_data[:,2] == k-1)*(input_data[:,3] == m-1))
                            for q in range(0,3):
                                p_ave[q,i,j,k,m] = np.sum((input_data[:,0] == i-1)*(input_data[:,1] == j-1)
                                    *(input_data[:,2] == k-1)*(input_data[:,3] == m-1)
                                    *(var_data == q-1))/n_count[i,j,k,m]
                        
            
        else:
            print('error -- too many inputs')
            return
            
        self.n_count = torch.tensor(n_count/self.n_data)
        self.prob_dist = torch.tensor(p_ave)
        
        return
    
    def sample(self,data_in=[]):
        
        if self.n_inputs == 0:
            p_temp = self.prob_dist
        elif self.n_inputs == 1:
            p_temp = torch.squeeze(self.prob_dist[data_in[0]+1,:])
        elif self.n_inputs == 2:
            p_temp = torch.squeeze(self.prob_dist[data_in[0]+1,data_in[1]+1,:])
        elif self.n_inputs == 3:
            p_temp = torch.squeeze(self.prob_dist[data_in[0]+1,data_in[1]+1,data_in[2]+1,:])
            
        else:
            print('error -- too many inputs')
            p_temp = []
        
        return torch.squeeze(pyro.sample(self.name,pyro.distributions.Categorical(probs = p_temp)).int()-1)

In [221]:
class cg_graph():
    
    def __init__(self,str_list=[],bel_graph=[]):
        
        edge_list = []

        entity_list = []
        
        if str_list:

            for item in str_list:

                sub_ind = item.find('=')

                sub_temp = item[:sub_ind-1]
                obj_temp = item[sub_ind+3:]
                
                rel_temp = item[sub_ind:sub_ind+2]

                if sub_temp not in entity_list:
                    entity_list.append(sub_temp)
                if obj_temp not in entity_list:
                    entity_list.append(obj_temp)

                edge_list.append([sub_temp,obj_temp,rel_temp])
                
        elif bel_graph:
            
            for item in bel_graph.edges:
                edge_temp = bel_graph.get_edge_data(item[0],item[1],item[2])
                sub_temp = str(item[0])
                obj_temp = str(item[1])
                rel_temp = edge_temp['relation']
                
                if sub_temp not in entity_list:
                    entity_list.append(sub_temp)
                if obj_temp not in entity_list:
                    entity_list.append(obj_temp)
                
                edge_list.append([sub_temp,obj_temp,rel_temp])

        n_nodes = len(entity_list)
        self.n_nodes = n_nodes

        adj_mat = np.zeros((n_nodes,n_nodes),dtype=int)

        for item in edge_list:
            out_ind = entity_list.index(item[0])
            in_ind = entity_list.index(item[1])
            adj_mat[out_ind,in_ind] = 1
            
        self.edge_list = edge_list
        self.entity_list = entity_list
        
        #self.graph = nx.DiGraph(adj_mat)
        
        node_dict = {}
        
        for i in range(0,n_nodes):
            node_dict[entity_list[i]] = cg_node(np.sum(adj_mat[:,i]),entity_list[i])
        
        self.node_dict = node_dict
        
        #self.parent_ind_list = []
        #self.child_ind_list = []
        self.parent_name_dict = {}
        #self.child_name_dict = {}
        
        self.parent_ind_list = [np.where(adj_mat[:,i] > 0)[0] for i in range(0,n_nodes)]
        #self.child_ind_list = [np.where(self.adj_mat[i,:] > 0)[0] for i in range(0,n_nodes)]
        
        for i in range(0,n_nodes):
            self.parent_name_dict[entity_list[i]] = [entity_list[item] for item in self.parent_ind_list[i]]
            #self.child_name_dict[entity_list[i]] = [entity_list[item] for item in self.child_ind_list[i]]

        return
    
    
    def prob_init(self,data_in):
        # initialize all of the nodes
        
        exog_list = []
        prob_dict = {}
        
        for name in self.node_dict:
            i = self.entity_list.index(name)
            data_in_temp = data_in[:,self.parent_ind_list[i]]
            data_out_temp = data_in[:,i]
            
            self.node_dict[name].p_init(data_in_temp,data_out_temp)
            
            if self.node_dict[name].n_inputs == 0:
                exog_list.append(name)
            prob_dict[name] = self.node_dict[name].prob_dist
        
        self.exog_list = exog_list
        self.prob_dict = prob_dict

        return
        
    def model_sample(self):
        
        # define exogenous samples
        
        sample_dict = {}
        
        for item in self.exog_list:
            sample_dict[item] = self.node_dict[item].sample()
            
        flag = 0
        while flag == 0:
            
            # find all nodes not in sample_dict with parents entirely in sample dict and sample those nodes
            for item in self.entity_list:
                if (item not in sample_dict 
                    and np.all([item2 in sample_dict for item2 in self.parent_name_dict[item]])):
                    
                    sample_dict[item] = self.node_dict[item].sample(
                        [sample_dict[item2] for item2 in self.parent_name_dict[item]])
            
            # if sample dict has all of the nodes in entity list, stop
            if sorted([item for item in sample_dict]) == sorted(self.entity_list):
                flag = 1
            
        
        return sample_dict
    
    def model_cond_sample(self,data_dict):
        
        data_in = {}
        for item in data_dict:
            data_in[item] = data_dict[item] + 1
        
        cond_model = pyro.condition(self.model_sample,data=data_in)
        return cond_model()
        
    def model_do_sample(self,do_dict):
        
        data_in = {}
        for item in do_dict:
            data_in[item] = do_dict[item] + 1
        
        do_model = pyro.do(self.model_sample,data=data_in)
        return do_model()
    
    def model_do_cond_sample(self,do_dict,data_dict):
        
        if np.any([[item1 == item2 for item1 in do_dict] for item2 in data_dict]):
            print('overlapping lists!')
            return
        else:
            do_dict_in = {}
            for item in do_dict:
                do_dict_in[item] = do_dict[item] + 1
                
            data_dict_in = {}
            for item in data_dict:
                data_dict_in[item] = data_dict[item] + 1
            
            do_model = pyro.do(self.model_sample,data=do_dict_in)
            cond_model = pyro.condition(do_model,data=data_dict_in)
            return cond_model()
    
    def model_counterfact(self,obs_dict,do_dict_counter):
        # find conditional distribution on exogenous variables given observations and do variable values
        cond_dict = self.model_cond_sample(obs_dict)
        cond_dict_temp = {}
        for item in self.exog_list:
            cond_dict_temp[item] = cond_dict[item]
        
        # evaluate observed variables given this condition distribution and do_dict_counter do-variables
        return self.model_do_cond_sample(do_dict_counter,cond_dict_temp)
        
        
        
        
  

In [222]:
str_list = ['temp =| cloudy','cloudy => rainy','temp => icream','rainy =| icream']
graph_test = cg_graph(str_list)

In [173]:
for item in graph_test.node_dict:
    print([graph_test.node_dict[item].name,graph_test.node_dict[item].n_inputs])

['temp', 0]
['cloudy', 1]
['rainy', 1]
['icream', 2]


In [223]:
graph_test.prob_init(tot_data)
print(graph_test.exog_list)

['temp']


In [ ]:
for item in graph_test.node_dict:
    print(item)
    print(graph_test.node_dict[item].n_count)
    print(graph_test.node_dict[item].prob_dist)
    print()

In [178]:
graph_test.model_sample()

{'cloudy': tensor(-1, dtype=torch.int32),
 'icream': tensor(0, dtype=torch.int32),
 'rainy': tensor(-1, dtype=torch.int32),
 'temp': tensor(1, dtype=torch.int32)}

In [200]:
cond_dict = {}
cond_dict['cloudy'] = torch.Tensor([-1]).int()

cond_test = graph_test.model_cond_sample(cond_dict)

for item in cond_test:
    print(item)
    print(cond_test[item])
    print()

temp
tensor(-1, dtype=torch.int32)

cloudy
tensor(-1, dtype=torch.int32)

rainy
tensor(1, dtype=torch.int32)

icream
tensor(1, dtype=torch.int32)



In [225]:
do_dict = {}
do_dict['rainy'] = torch.Tensor([1]).int()

do_test = graph_test.model_do_sample(do_dict)

for item in do_test:
    print(item)
    print(do_test[item])
    print()



temp
tensor(0, dtype=torch.int32)

cloudy
tensor(1, dtype=torch.int32)

rainy
tensor(1, dtype=torch.int32)

icream
tensor(0, dtype=torch.int32)



In [226]:
do_cond_test = graph_test.model_do_cond_sample(do_dict,cond_dict)
for item in do_cond_test:
    print(item)
    print(do_cond_test[item])
    print()

temp
tensor(0, dtype=torch.int32)

cloudy
tensor(-1, dtype=torch.int32)

rainy
tensor(1, dtype=torch.int32)

icream
tensor(1, dtype=torch.int32)



In [263]:
obs_dict = {}
obs_dict['icream'] = torch.Tensor([1]).int()
obs_dict['rainy'] = torch.Tensor([-1]).int()

do_dict = {}
do_dict['rainy'] = torch.Tensor([0]).int()


counter_test = graph_test.model_counterfact(obs_dict,do_dict)
for item in counter_test:
    print(item)
    print(counter_test[item])
    print()


temp
tensor(-1, dtype=torch.int32)

cloudy
tensor(1, dtype=torch.int32)

rainy
tensor(0, dtype=torch.int32)

icream
tensor(1, dtype=torch.int32)



In [ ]:
graph_test.calc_do_cond(['rainy'],do_dict,['temp'],['icream'])

In [ ]:
for item in graph_test.prob_dict:
    print(item)
    print(graph_test.prob_dict[item])
    print()

In [ ]:
x = pyro.sample('',pyro.distributions.Multinomial(probs = torch.Tensor([0.3,0.2,0.5]))).bool()
#samp_out = pyro.sample('',pyro.distributions.Multinomial(probs = self.prob_dist))
print(x)
y = torch.Tensor([-1,0,1])[x]
print(y)

z = torch.Tensor([1]).int()
print(z)
print(z+1)

In [ ]:
print(graph_test.calc_prob(['icream']))

In [ ]:
print(dir(graph_test))
print()
print(graph_test.entity_list)
print(graph_test.adj_mat)

In [8]:
def indep_vars(n_samples):
    
    T_list = []
    C_list = []
    P_list = []
    
    for i in range(0,n_samples):
        
        #x = pyro.sample("x_{}".format(i), pyro.distributions.Normal(20,5))
        
        #T_temp = pyro.distributions.Normal(20,5).sample()
        #C_temp = 0.5*pyro.distributions.Beta(1,1+T_temp/10).sample() + 0.5*pyro.distributions.Uniform(0,1).sample()
        #P_temp = (0.5*pyro.distributions.Exponential(1).sample() 
            #+ 0.5*pyro.distributions.Exponential(1/(C_temp+1)).sample())
        
        T_list.append(pyro.sample("T_{}".format(i), pyro.distributions.Normal(20,5)))
        
        C_list.append(0.5*pyro.sample("C1_{}".format(i),pyro.distributions.Beta(1,1+T_list[-1]/10)) 
            + 0.5*pyro.sample("C2_{}".format(i),pyro.distributions.Uniform(0,1)))
        P_list.append(0.5*pyro.sample("P1_{}".format(i), pyro.distributions.Exponential(1))
            + 0.5*pyro.sample("P2.{}".format(i),pyro.distributions.Exponential(1/(C_list[-1]+1))))
        
    return T_list,C_list,P_list

def dep_vars(T_list,C_list,P_list):
    
    n_pts = len(T_list)
    
    I_list = []
    
    for i in range(0,n_pts):
        
        T_temp = T_list[i]
        C_temp = C_list[i]
        P_temp = P_list[i]
        
        if P_temp > 2.5 or T_temp < 15:
            I_list.append(1e-6*pyro.sample("I_{}".format(i),pyro.distributions.Bernoulli(1)))
        else:
            I_list.append(pyro.sample("I_{}".format(i),
                pyro.distributions.Beta(2*(2.5-P_temp)*(T_temp-12)/(2.5*12),2)))
            #I_temp = torch.tensor(0.5)
        
    return I_list

In [9]:
n_data = 10000
temp,cloud,precip = indep_vars(n_data)
icream = dep_vars(temp,cloud,precip)

In [115]:
# trinarize data relative to baseline
T_base = 20
C_base = 0.38
P_base = 1.2
I_base = 0.23

T_sig = 1.0
C_sig = 0.02
P_sig = 0.06
I_sig = 0.01

n_count_tri = np.zeros((3,3,3))
p_ave_tri = np.zeros((3,3,3,3))

def cond_test(val,base,sig):
    
    conds = [val < base-sig,val > base-sig and val < base+sig,val > base+sig]
    
    return conds.index(True)-1
        
T_ind = []
C_ind = []
P_ind = []
I_ind = []
for ind in range(0,n_data):
    T_ind.append(cond_test(temp[ind],T_base,T_sig))
    C_ind.append(cond_test(cloud[ind],C_base,C_sig))
    P_ind.append(cond_test(precip[ind],P_base,P_sig))
    I_ind.append(cond_test(icream[ind],I_base,I_sig))

In [116]:
tot_data = np.asarray([T_ind,C_ind,P_ind,I_ind]).T

In [12]:
print(np.shape(tot_data))

(10000, 4)


In [ ]:
print(tot_data[0:5,:])